In [13]:
## This ipynb file is made up by 6 cells
# first cell is the methods will be used in this classifier program and please change nothing with it
# second cell is the test cases. By using simple set of methods I to mkae apresentation of the methods in the first component
# the third cell is the KNN validition. In the validition, In this validition, the data input is needed. You can change the valid set by changing peremeter.
# the fourth cell is the KNN classifier. In this part, the data input is needed and the result will be out put.
# the fifth cell is the Naive Bayes validition. In the validition, In this validition, the data input is needed. You can change the valid set by changing peremeter.
# the sixth cell is the Naive Bayes classifier. In this part, the data input is needed and the result will be out put.

In [4]:
# hardware specifications:
# cpu: AMD Ryzen 7 1700 Eight-Core Processor
# memory: 8.00 GB 
# System: 64-bit based on X64
# software specifications:
# Windows 10 pro version: 1909
# Python version: 3.8.1
# jupyter notebook version: 6.1.4

In [1]:
# method cell
import h5py
import numpy as np
import os
import matplotlib.pyplot as pl
def load_data():
    #the data from the input class will be loaded into 4 ndarray objects and returned
    
    with h5py.File('./Input/train/images_training.h5','r') as H:
        data_train = np.copy(H['datatrain'])
    with h5py.File('./Input/train/labels_training.h5','r') as H:
        label_train = np.copy(H['labeltrain'])
    with h5py.File('./Input/test/images_testing.h5','r') as H:
        data_test = np.copy(H['datatest'])
    with h5py.File('./Input/test/labels_testing_2000.h5','r') as H:
        label_test = np.copy(H['labeltest'])    
   
    return data_train,label_train,data_test,label_test
    

def PCA(image,component):   
    # PCA algorithm for the individual image
    #the image input into PCA method will be reconstructed and return
    # due to the original image size from the train data is 28*28, which is not so large
    # the component to train should be small
    U, s, Vt = np.linalg.svd(image, full_matrices=False)
    S = np.diag(s)
    image_reconstructed = U[0:U.shape[0], 0:component]\
           .dot(S[0:component,0:component])\
           .dot(Vt[0:component, 0:Vt.shape[1]])
    # compress with SVD 
    return image_reconstructed

def PCA_set(image_set,component):
    # PCA algorithm for the set of iamges (database)
    image_set = image_set.reshape((image_set.shape[0], 28, 28))
    # the images need to be converted firstly for being compressed
    for i in range(image_set.shape[0]):
        image_set[i] = PCA(image_set[i],component)
    # one image in the set will be compressed in one time
    image_set = image_set.reshape(image_set.shape[0], -1)
    # after the images were comporessed, the set will be reshaped to ndarray
    return image_set

# KNN related algorithms here
def KNN_algorithm(data_test, data_train, label_train, k):
    # calculate the  Euclidean distance
    distance = (((np.tile(data_test, (data_train.shape[0])).reshape((30000, 784)) - data_train )**2).sum(axis=1))**0.5
    # set up 10 classes for prediction count record
    classes = np.zeros((10), np.int32)  
    # stalistic the number of the nearaest array of the first k classses
    for i in range(k): 
        voteIlabel = label_train[distance.argsort()[i]]
        classes[voteIlabel] += 1
    max = 0
    prediction = 0
    # compare the nearest classes and return the max as the prediction
    for i in range(classes.shape[0]):
        if classes[i] >= max:
            max = classes[i]
            prediction = i
    return prediction

# Naive Bayes related algorithms here
def normalize_set(images):
    # the normalization algorithm will reload the gray scale images into black-white
    # by the way the images' np array will become filled with 0 and 1
    m = images.shape[0]
    n = np.array(images).shape[1]
    for i in range(m):
        for j in range(n):
            if images[i, j] != 0:
                images[i, j] = 1
            else:
                images[i, j] = 0
    return images

def cal_probability(img, label, prior_probability, conditional_probability):
    # this method will be used to calculate the probability of every test image \n
    # with the prior probability and conditional probability of the training data
    probability = int(prior_probability[label]) 
    n = img.shape[0]
    for i in range(n): 
        probability *= int(conditional_probability[label][i][img[i].astype(int)])
    return probability

def Bayes(data_train,label_train,data_test,class_num):
    # the data inputed should be normalized
    # peremeters setup
    # prior_probability and conditional_probability will contains the probibilities of different classs and features of the taining class 
    shape = data_train.shape[0]
    features = data_train.shape[1]
    prior_probability = np.zeros(class_num)  
    conditional_probability = np.zeros((class_num, features, 2))  
    
    
    #calculate the prior probability and conditional probability from the training data
    for i in range(shape):  
        images = data_train[i]  
        label = label_train[i]  
        # statistic the number of the labels of diferent classses
        # prior_probability/features = the prior prebability of a single classs from the taining data
        prior_probability[label] += 1  
        for j in range(features):  
            temp = images[j].astype(int) 
            # the conditional probability will statistic the 1 and 0 in the features of a label(class) 
            conditional_probability[label][j][temp] += 1
    # calculate the specific conditional proability for single class
    for i in range(class_num):
        for j in range(features):
            pix_0 = conditional_probability[i][j][0]
            pix_1 = conditional_probability[i][j][1]
            # calsulate the o and 1 at the spefic position in the np array probability
            probability_0 = (float(pix_0) / float(pix_0 + pix_1)) * 10000 + 1
            probability_1 = (float(pix_1) / float(pix_0 + pix_1)) * 10000 + 1
            conditional_probability[i][j][0] = probability_0
            conditional_probability[i][j][1] = probability_1        
    
    # calsulate the probability of the test image
    predictions = []
    test_shape = data_test.shape[0]
    for i in range(test_shape):
        img = np.array(data_test[i])
        # setup the max label for prediction result
        max_label = 0
        max_probability = cal_probability(img, 0, prior_probability, conditional_probability)
        
        for j in range(1, 10):  
            probability = cal_probability(img, j, prior_probability, conditional_probability)
            if max_probability < probability:
                max_probability = probability
                max_label = j
        predictions.append(max_label) 
    
    return predictions    
    
def save_result(prediction_list):
    # this method will be used to save the result to the output file
    result = np.array(prediction_list)
    with h5py.File('./Output/predicted_labels.h5','w') as H:
        H.create_dataset('Output',data=result)
    
    

In [3]:
# test cell
def load_test():
    print("Start loading data...")
    print("...")
    a,b,c,d = load_data()
    print("loading successfully!")
    print("The shape of the training data is ",a.shape)
    print("test finished\n")
    
def PCA_test():
    print("PCA test start...")
    print("...")
    PCA_component = 4
    a,b,c,d = load_data()
    test = a.reshape((a.shape[0], 28, 28))
    pl.figure(figsize=(15,10)) #figsize=(15,10)
    pl.subplot(121)
    pl.imshow(test[0], cmap=pl.cm.gray)
    pl.title('Original image')
    test1 = PCA_set(a,PCA_component)
    test1 = test1.reshape((a.shape[0], 28, 28))
    pl.subplot(122)
    pl.imshow(test1[0], cmap=pl.cm.gray)
    pl.title('Compressed image')
    pl.show()
    print("test finished\n")
    
def normalize_test():
    print("normalizing test start...")
    print("...")
    a,b,c,d = load_data()
    test = a.reshape((a.shape[0], 28, 28))
    pl.figure(figsize=(15,10)) #figsize=(15,10)
    pl.subplot(121)
    pl.imshow(test[0], cmap=pl.cm.gray)
    pl.title('Original image')
    test1 = normalize_set(a)
    test1 = test1.reshape((a.shape[0], 28, 28))
    pl.subplot(122)
    pl.imshow(test1[0], cmap=pl.cm.gray)
    pl.title('normalized image')
    pl.show()
    print("test finished\n")
    
def read_saved_prediction():
    print("read result test start...")
    print("...")
    with h5py.File('./Output/predicted_labels.h5','r') as H:
        prediction_result = np.copy(H['Output'])
    print("The shape of the result data is ",prediction_result.shape)
    print("test finished\n")
load_test()
PCA_test()
normalize_test()
read_saved_prediction()

read result test start...
...
The shape of the result data is  (5000,)
test finished



In [6]:
# KNN Validation cell
import time
from datetime import datetime
def KNN_Validition():
    print("KNN validition start up")
    print("...")
    # record the satrt time
    PCA_component = 5
    a = datetime.now()
    data_train,label_train,data_test,label_test=load_data()
    data_test = data_train[0:1000]
    data_train = PCA_set(data_train,PCA_component)
    data_test = PCA_set(data_test,PCA_component)
    # you can change the retio of the test to control the test limited
    # ttestRatio should be between 0 to 1
    testRatio = 1
    # The k of the nearist training data to calculate
    # You can change the k 
    K = 3
    # the classify time will increase by k increasing
    train_row = data_train.shape[0]  
    test_row = data_test.shape[0]
    testNum = int(test_row * testRatio)
    predictions = []
    for i in range(testNum):
        result = KNN_algorithm(data_test[i],data_train,label_train,K)
        predictions.append(result)
    correct = 0
    for i in range(testNum):
        print("The %d test, the prediction is %s and the truth is %s"%(i,predictions[i],label_train[i]) )
        if(predictions[i]==label_train[i]):
            correct +=1
    accuracy = correct/(1000*testRatio)
    print("The accuracy is %s"%accuracy)
    #record the finish time    
    b = datetime.now()  
    durn = (b-a).seconds
    print("classify finished, %d seconds used"%durn)
KNN_Validition()    

KNN validition start up
...
The 0 test, the prediction is 6 and the truth is 6
The 1 test, the prediction is 0 and the truth is 0
The 2 test, the prediction is 1 and the truth is 1
The 3 test, the prediction is 2 and the truth is 2
The 4 test, the prediction is 9 and the truth is 9
The 5 test, the prediction is 7 and the truth is 5
The 6 test, the prediction is 9 and the truth is 9
The 7 test, the prediction is 9 and the truth is 7
The 8 test, the prediction is 6 and the truth is 6
The 9 test, the prediction is 2 and the truth is 2
The accuracy is 0.8
classify finished, 7 seconds used


In [ ]:
# KNN classifier cell
import time
from datetime import datetime
def KNN_classifier():
    print("KNN classifier start up")
    print("...")
    # record the satrt time
    a = datetime.now()
    # We set the component to retain as 4, which works good by KNN
    PCA_component = 5
    data_train,label_train,data_test,label_test=load_data()
    data_train = PCA_set(data_train,PCA_component)
    data_test = PCA_set(data_test,PCA_component)
    
    # you can change the retio of the test to control the test limited
    # testRatio should be between 0 to 1
    testRatio = 1
    # The k of the nearist training data to calculate
    # You can change the k but the classify time will increase by k increasing
    K = 3
    train_row = data_train.shape[0]  
    test_row = data_test.shape[0]
    testNum = int(test_row * testRatio)
    predictions = []
    for i in range(testNum):
        result = KNN_algorithm(data_test[i],data_train,label_train,K)
        predictions.append(result)
    save_result(predictions)
    #record the finish time    
    b = datetime.now()  
    durn = (b-a).seconds
    correct = 0
    for i in range(len(predictions)):
        if(predictions[i]==label_test[i]):
            correct += 1
    print("accuracy = %s"%(correct/len(predictions)))
    print("classify finished, %d seconds used"%durn)
KNN_classifier()    

KNN classifier start up
...


In [34]:
# Naive Bayes Validition cell
import time
from datetime import datetime
def Naive_Bayes_Validition():
    print("Naive Bayes validition start up")
    print("...")
    # record the satrt time
    a = datetime.now()
    data_train,label_train,data_test,label_test=load_data()
    data_test = data_train[0:1000]
    #data_train = PCA_set(data_train)
    #data_test = PCA_set(data_test)
    data_train = normalize_set(data_train)
    data_test = normalize_set(data_test)
    predictions = Bayes(data_train,label_train,data_test,10)
    correct = 0
    for i in range(1000):
        print("The %d test, the prediction is %s and the truth is %s"%(i,predictions[i],label_train[i]) )
        if(predictions[i]==label_train[i]):
            correct +=1
    accuracy = correct/1000
    print("The accuracy is %s"%accuracy)
    #record the finish time    
    b = datetime.now()  
    durn = (b-a).seconds
    print("classify finished, %d seconds used"%durn)
Naive_Bayes_Validition()

Naive Bayes validition start up
...
The 0 test, the prediction is 0 and the truth is 6
The 1 test, the prediction is 3 and the truth is 0
The 2 test, the prediction is 1 and the truth is 1
The 3 test, the prediction is 4 and the truth is 2
The 4 test, the prediction is 9 and the truth is 9
The 5 test, the prediction is 7 and the truth is 5
The 6 test, the prediction is 9 and the truth is 9
The 7 test, the prediction is 9 and the truth is 7
The 8 test, the prediction is 3 and the truth is 6
The 9 test, the prediction is 2 and the truth is 2
The 10 test, the prediction is 3 and the truth is 3
The 11 test, the prediction is 0 and the truth is 6
The 12 test, the prediction is 8 and the truth is 8
The 13 test, the prediction is 2 and the truth is 2
The 14 test, the prediction is 8 and the truth is 8
The 15 test, the prediction is 3 and the truth is 1
The 16 test, the prediction is 2 and the truth is 2
The 17 test, the prediction is 7 and the truth is 7
The 18 test, the prediction is 2 and t

In [22]:
#Naive Bayes classifier cell
import time
from datetime import datetime
def Naive_Bayes_classifier():
    print("Naive Bayes classifier start up")
    print("...")
    # record the satrt time
    a = datetime.now()
    # We set the component to retain as 28, which works good by KNN
    PCA_component = 28
    data_train,label_train,data_test,label_test=load_data()
    #data_train = PCA_set(data_train,PCA_component)
    #data_test = PCA_set(data_test,PCA_component)
    data_train = normalize_set(data_train)
    data_test = normalize_set(data_test)
    # you can change the retio of the test to control the test limited
    # ttestRatio should be between 0 to 1
    ratio = 1
    end = int(5000* ratio)
    data_test = data_test[0:end]
    predictions = Bayes(data_train,label_train,data_test,10)
    save_result(predictions)
    #record the finish time    
    b = datetime.now()  
    durn = (b-a).seconds
    correct = 0
    for i in range(len(predictions)):
        if(predictions[i]==label_test[i]):
            correct += 1
    print("accuracy = %s"%(correct/len(predictions)))
    print("classify finished, %d seconds used"%durn)
Naive_Bayes_classifier()

Naive Bayes classifier start up
...
accurate = 0.62
classify finished, 134 seconds used
